In [ ]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
df = pd.read_csv("annual_membership_expirations_2024.csv")

In [ ]:
df.drop("Unnamed: 7", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
counts_by_month = df.groupby([pd.to_datetime(df["Membership Expiration Date"]).dt.month])[["Account ID"]].count()

indexes = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
    }

counts_by_month.rename(columns={"Account ID": "Count"}, index=indexes, inplace=True)

In [ ]:

sns.barplot(x=counts_by_month.index, y=counts_by_month["Count"])

plt.xlabel("Month")
plt.title("Annual Membership Expirations by Month in 2024")

plt.savefig("annual_membership_expirations_2024.png", dpi=300)


In [ ]:
import requests, base64

from config import N_APIkey, N_APIuser

# Neon Account Info
N_AUTH = f"{N_APIuser}:{N_APIkey}"
N_BASE_URL = "https://api.neoncrm.com/v2/"
N_SIGNATURE = base64.b64encode(bytearray(N_AUTH.encode())).decode()
N_HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Basic {N_SIGNATURE}",
}

df = pd.read_csv("annual_memberships_april_may_2023.csv")

df.drop("Unnamed: 7", axis=1, inplace=True)

In [ ]:
df.insert(loc=len(df.columns), column="previous_membership", value=False)
df.insert(loc=len(df.columns), column="previous_membership_type", value=np.nan)

for column in ["Membership Start Date", "Membership Enrollment Date", "Membership Expiration Date"]:
    df[column] = pd.to_datetime(df[column])

df.head()

In [ ]:
session = requests.Session()
session.headers.update(N_HEADERS)

def get_account_membership_previous_to_date(neonId: int, membership_start_date: datetime.datetime) -> bool:
    base_url = "https://api.neoncrm.com/v2"
    resource_path = f"/accounts/{neonId}/memberships"

    url = base_url + resource_path

    acct_memberships = session.get(url)

    if acct_memberships.status_code != 200:
        print(acct_memberships.status_code, neonId)
    
    acct_memberships = acct_memberships.json()["memberships"]

    prev_mem = False
    prev_mem_type = np.nan

    if len(acct_memberships) < 2:
        return {"previous_membership": prev_mem, "mem_type": prev_mem_type}
    
    for membership in acct_memberships:
        start_date = datetime.datetime.fromisoformat(membership["termStartDate"])
        end_date = datetime.datetime.fromisoformat(membership["termEndDate"])
        if  end_date == membership_start_date - datetime.timedelta(days=1):
            prev_mem_type = "annual" if membership["termUnit"] == "YEAR" else "monthly"
            prev_mem = True
            break

    return {"previous_membership": prev_mem, "mem_type": prev_mem_type}

In [ ]:
for account in df.itertuples():

    prev_mem_values = get_account_membership_previous_to_date(account[df.columns.get_loc("Account ID")+1], account[df.columns.get_loc("Membership Start Date")+1])

    df.loc[account.Index, "previous_membership"] = prev_mem_values["previous_membership"]
    df.loc[account.Index, "previous_membership_type"] = prev_mem_values["mem_type"]

In [ ]:
df["previous_membership_type"].fillna("n/a", inplace=True)

df.head()

In [ ]:
counts = df.groupby(["previous_membership", "previous_membership_type"], as_index=False)[["Account ID"]].count()
counts.rename(columns={"Account ID": "count"}, inplace=True)

In [ ]:
counts.to_csv("april_may_counts.csv")

In [ ]:
df.insert(len(df.columns), "auto_renew", False)

In [ ]:
for row in df.itertuples():

    

    df.loc